# Restricted Two Body Problem: Elliptical Orbits Around a Central Mass

This is the general case of a Keperian orbit.<br>
A light body (e.g. a planet) orbits a heavy central body (e.g. the sun).  The orbit is an ellipse with the primary at one focus.

In [1]:
# Library imports
import tensorflow as tf
import rebound
import numpy as np
import matplotlib.pyplot as plt
import time

# Aliases
keras = tf.keras

In [2]:
# Local imports
from utils import load_vartbl, save_vartbl, plot_style
from tf_utils import gpu_grow_memory
from tf_utils import Identity
from r2b import VectorError
from orbital_element import make_data_orb_elt, make_dataset_elt_to_cfg, make_dataset_cfg_to_elt
from orbital_element import OrbitalElementToConfig, make_model_elt_to_cfg

In [3]:
# Grow GPU memory (must be first operation in TF)
gpu_grow_memory()

In [4]:
# Plot style 
plot_style()

In [5]:
# Lightweight serialization
fname = '../pickle/orbital_element.pickle'
vartbl = load_vartbl(fname)

In [6]:
# Create small data set for orbital elements; dictionaries of numpy arrays
n = 100
a_min = 1.0
a_max = 2.0
e_max = 1.0
inc_max = np.pi/4.0
seed=42
elts, cart = make_data_orb_elt(n=n, a_min=a_min, a_max=a_max, e_max=e_max, inc_max=inc_max, seed=seed)

In [7]:
# Create a tensorflow Dataset instance in both directions
batch_size = 64
ds_e2c = make_dataset_elt_to_cfg(n=n, a_min=a_min, a_max=a_max, e_max=e_max, 
                                 inc_max=inc_max, seed=seed, batch_size=batch_size)
ds_c2e = make_dataset_cfg_to_elt(n=n, a_min=a_min, a_max=a_max, e_max=e_max, 
                                 inc_max=inc_max, seed=seed, batch_size=batch_size)

In [8]:
# Example batch
elts, cart = list(ds_e2c.take(1))[0]

# Unpack orbital elements
a = elts['a']
e = elts['e']
inc = elts['inc']
Omega = elts['Omega']
omega = elts['omega']
f = elts['f']
mu = elts['mu']

# Unpack cartesian coordinates
q = cart['q']
v = cart['v']

# Review shapes
print(f'Example batch sizes:')
print(f'a    = {a.shape}')
print(f'e    = {e.shape}')
print(f'inc  = {inc.shape}')
print(f'Omega= {Omega.shape}')
print(f'omega= {omega.shape}')
print(f'f    = {f.shape}')
print(f'mu   = {f.shape}')
print(f'q    = {q.shape}')
print(f'v    = {v.shape}')

Example batch sizes:
a    = (64,)
e    = (64,)
inc  = (64,)
Omega= (64,)
omega= (64,)
f    = (64,)
mu   = (64,)
q    = (64, 3)
v    = (64, 3)


In [9]:
# Run the layer on the batch of orbital elements
inputs_e2c = (a, e, inc, Omega, omega, f, mu)
cart_rec = OrbitalElementToConfig()(inputs_e2c)

In [10]:
# Create a model mapping orbital elements to cartesian coordinates
model_e2c = make_model_elt_to_cfg()

In [11]:
# Inputs to compile this model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'q': VectorError(name='q_loss'),
        'v': VectorError(name='v_loss')}

metrics = None

loss_weights = {'q': 1.0,
                'v': 1.0}

In [12]:
# Compile the e2c model
model_e2c.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [13]:
# Verify that model matches rebound
model_e2c.evaluate(ds_e2c)

2/2 [==============================] - 0s 112ms/step - loss: 3.0719e-14 - q_loss: 2.0408e-14 - v_loss: 1.0310e-14


[3.071855954556255e-14, 2.0408156e-14, 1.0310403e-14]

In [14]:
# Summary of the model mapping orbital elements to position
# model_e2c.summary()

In [63]:
class ConfigToOrbitalElement(keras.layers.Layer):
    def call(self, inputs):
        """Compute orbital elements (a, e, inc, Omega, omega, f) from configuration (qx, qy, qz, vx, vy, vz, mu)"""
        # Unpack inputs
        qx, qy, qz, vx, vy, vz, mu = inputs

        # See rebound library tools.c, reb_tools_particle_to_orbit_err
        
        # The distance from the primary
        r = tf.sqrt(tf.square(qx) + tf.square(qy) + tf.square(qz))
        
        # The speed and its square
        v2 = tf.square(vx) + tf.square(vy) + tf.square(vz)
        v = tf.sqrt(v2)
        
        # The speed squared of a circular orbit
        v2_circ = mu / r
        
        # The semi-major axis
        a = -mu / (v2 - tf.constant(2.0) * v2_circ)
        
        # The specific angular momentum vector and its magnitude
        hx = qy*vz - qz*vy
        hy = qz*vx - qx*vz
        hz = qx*vy - qy*vx
        h = tf.sqrt(tf.square(hx) + tf.square(hy) + tf.square(hz))
        
        # The excess squared speed vs. a circular orbit
        v2_diff = v2 - v2_circ
        
        # The dot product of v and r
        vr = qx * vx + qy*vy + qz*vz
        # r times v dot r
        rvr = r * vr
        # Inverse of mu
        mu_inv = tf.constant(1.0) / mu
        
        # Eccentricity vector
        ex = mu_inv * (v2_diff * qx - rvr * vx)
        ey = mu_inv * (v2_diff * qy - rvr * vy)
        ez = mu_inv * (v2_diff * qz - rvr * vz)
        # The eccentricity is the magnitude of this vector
        e = tf.sqrt(tf.square(ex) + tf.square(ey) + tf.square(ez))
        
        # Dummy placeholders for rest
        inc = tf.constant(0.0) * a
        Omega = tf.constant(0.0) * a
        omega = tf.constant(0.0) * a
        f = tf.constant(0.0) * a
        
        return a, e, inc, Omega, omega, f

    def get_config(self):
        return dict()

In [64]:
def make_model_cfg_to_elt():
    """Model that transforms from orbital elements to cartesian coordinates"""
    # Create input layers    
    q = keras.Input(shape=(3,), name='q')
    v = keras.Input(shape=(3,), name='v')
    mu = keras.Input(shape=(1,), name='mu')
    
    # Wrap these up into one tuple of inputs for the model
    inputs_model = (q, v, mu,)
    
    # Unpack coordinates from inputs
    qx = keras.layers.Reshape(target_shape=(1,), name='qx')(q[:,0])
    qy = keras.layers.Reshape(target_shape=(1,), name='qy')(q[:,1])
    qz = keras.layers.Reshape(target_shape=(1,), name='qz')(q[:,2])
    vx = keras.layers.Reshape(target_shape=(1,), name='vx')(v[:,0])
    vy = keras.layers.Reshape(target_shape=(1,), name='vy')(v[:,1])
    vz = keras.layers.Reshape(target_shape=(1,), name='vz')(v[:,2])

    # Tuple of inputs for the layer
    inputs_layer = (qx, qy, qz, vx, vy, vz, mu,)

    # Calculations are in one layer that does all the work...
    a, e, inc, Omega, omega, f = ConfigToOrbitalElement(name='config_to_orbital_element')(inputs_layer)

    # Name the outputs of the layer
    a = Identity(name='a')(a)
    e = Identity(name='e')(e)
    inc = Identity(name='inc')(inc)
    Omega = Identity(name='Omega')(Omega)
    omega = Identity(name='omega')(omega)
    f = Identity(name='f')(f)

    # Wrap up the outputs
    outputs = (a, e, inc, Omega, omega, f)

    # Create a model from inputs to outputs
    model = keras.Model(inputs=inputs_model, outputs=outputs, name='cartesian_to_orbital_element')
    return model

In [65]:
# Run the layer on the batch of orbital elements
qx = q[:,0:1]
qy = q[:,1]
qz = q[:,2]
vx = v[:,0]
vy = v[:,1]
vz = v[:,2]
inputs_c2e = (qx, qy, qz, vx, vy, vz, mu)
elt_rec = ConfigToOrbitalElement()(inputs_c2e)

In [66]:
# Create a model mapping cartesian coordinates to orbital elements
model_c2e = make_model_cfg_to_elt()

In [67]:
# Inputs to compile the c2e model
optimizer = keras.optimizers.Adam(learning_rate=1.0E-3)

loss = {'a': keras.losses.MeanSquaredError(),
        'e': keras.losses.MeanSquaredError(),
        'inc': keras.losses.MeanSquaredError(),
        'Omega': keras.losses.MeanSquaredError(),
        'omega': keras.losses.MeanSquaredError(),
        'f': keras.losses.MeanSquaredError(),
       }

metrics = None

loss_weights = {'a': 1.0,
                'e': 1.0,
                'inc': 1.0,
                'Omega': 1.0,
                'omega': 1.0,
                'f': 1.0}

In [68]:
# model_c2e.summary()

In [69]:
# Compile the c2e model
model_c2e.compile(optimizer=optimizer, loss=loss, metrics=metrics, loss_weights=loss_weights)

In [70]:
model_c2e.evaluate(ds_c2e)

2/2 [==============================] - 0s 189ms/step - loss: 10.9354 - a_loss: 8.7598e-13 - e_loss: 0.0341 - inc_loss: 0.2254 - Omega_loss: 3.2830 - omega_loss: 3.9055 - f_loss: 3.4873


[10.935360431671143,
 8.759783e-13,
 0.034139823,
 0.22539008,
 3.2830186,
 3.9055257,
 3.4872866]